# What's this PyTorch business?

You've written a lot of code in this assignment to provide a whole host of neural network functionality. Dropout, Batch Norm, and 2D convolutions are some of the workhorses of deep learning in computer vision. You've also worked hard to make your code efficient and vectorized.

For the last part of this assignment, though, we're going to leave behind your beautiful codebase and instead migrate to one of two popular deep learning frameworks: in this instance, PyTorch (or TensorFlow, if you switch over to that notebook). 

### What is PyTorch?

PyTorch is a system for executing dynamic computational graphs over Tensor objects that behave similarly as numpy ndarray. It comes with a powerful automatic differentiation engine that removes the need for manual back-propagation. 

### Why?

* Our code will now run on GPUs! Much faster training. When using a framework like PyTorch or TensorFlow you can harness the power of the GPU for your own custom neural network architectures without having to write CUDA code directly (which is beyond the scope of this class).
* We want you to be ready to use one of these frameworks for your project so you can experiment more efficiently than if you were writing every feature you want to use by hand. 
* We want you to stand on the shoulders of giants! TensorFlow and PyTorch are both excellent frameworks that will make your lives a lot easier, and now that you understand their guts, you are free to use them :) 
* We want you to be exposed to the sort of deep learning code you might run into in academia or industry.

### PyTorch versions
This notebook assumes that you are using **PyTorch version 0.4**. Prior to this version, Tensors had to be wrapped in Variable objects to be used in autograd; however Variables have now been deprecated. In addition 0.4 also separates a Tensor's datatype from its device, and uses numpy-style factories for constructing Tensors rather than directly invoking Tensor constructors.

## How will I learn PyTorch?

Justin Johnson has made an excellent [tutorial](https://github.com/jcjohnson/pytorch-examples) for PyTorch. 

You can also find the detailed [API doc](http://pytorch.org/docs/stable/index.html) here. If you have other questions that are not addressed by the API docs, the [PyTorch forum](https://discuss.pytorch.org/) is a much better place to ask than StackOverflow.


# Table of Contents

This assignment has 5 parts. You will learn PyTorch on different levels of abstractions, which will help you understand it better and prepare you for the final project. 

1. Preparation: we will use CIFAR-10 dataset.
2. Barebones PyTorch: we will work directly with the lowest-level PyTorch Tensors. 
3. PyTorch Module API: we will use `nn.Module` to define arbitrary neural network architecture. 
4. PyTorch Sequential API: we will use `nn.Sequential` to define a linear feed-forward network very conveniently. 
5. CIFAR-10 open-ended challenge: please implement your own network to get as high accuracy as possible on CIFAR-10. You can experiment with any layer, optimizer, hyperparameters or other advanced features. 

Here is a table of comparison:

| API           | Flexibility | Convenience |
|---------------|-------------|-------------|
| Barebone      | High        | Low         |
| `nn.Module`     | High        | Medium      |
| `nn.Sequential` | Low         | High        |

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


# Part II. Barebones PyTorch

PyTorch ships with high-level APIs to help us define model architectures conveniently, which we will cover in Part II of this tutorial. In this section, we will start with the barebone PyTorch elements to understand the autograd engine better. After this exercise, you will come to appreciate the high-level model API more.

We will start with a simple fully-connected ReLU network with two hidden layers and no biases for CIFAR classification. 
This implementation computes the forward pass using operations on PyTorch Tensors, and uses PyTorch autograd to compute gradients. It is important that you understand every line, because you will write a harder version after the example.

When we create a PyTorch Tensor with `requires_grad=True`, then operations involving that Tensor will not just compute values; they will also build up a computational graph in the background, allowing us to easily backpropagate through the graph to compute gradients of some Tensors with respect to a downstream loss. Concretely if x is a Tensor with `x.requires_grad == True` then after backpropagation `x.grad` will be another Tensor holding the gradient of x with respect to the scalar loss at the end.

### PyTorch Tensors: Flatten Function
A PyTorch Tensor is conceptionally similar to a numpy array: it is an n-dimensional grid of numbers, and like numpy PyTorch provides many functions to efficiently operate on Tensors. As a simple example, we provide a `flatten` function below which reshapes image data for use in a fully-connected neural network.

Recall that image data is typically stored in a Tensor of shape N x C x H x W, where:

* N is the number of datapoints
* C is the number of channels
* H is the height of the intermediate feature map in pixels
* W is the height of the intermediate feature map in pixels

This is the right way to represent the data when we are doing something like a 2D convolution, that needs spatial understanding of where the intermediate features are relative to each other. When we use fully connected affine layers to process the image, however, we want each datapoint to be represented by a single vector -- it's no longer useful to segregate the different channels, rows, and columns of the data. So, we use a "flatten" operation to collapse the `C x H x W` values per representation into a single long vector. The flatten function below first reads in the N, C, H, and W values from a given batch of data, and then returns a "view" of that data. "View" is analogous to numpy's "reshape" method: it reshapes x's dimensions to be N x ??, where ?? is allowed to be anything (in this case, it will be C x H x W, but we don't need to specify that explicitly). 

In [4]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('Before flattening shape: ', x.shape)
    print('After flattening: ', flatten(x))
    print('After flattening shape: ', flatten(x).shape)

test_flatten()

Before flattening:  tensor([[[[  0.,   1.],
          [  2.,   3.],
          [  4.,   5.]]],


        [[[  6.,   7.],
          [  8.,   9.],
          [ 10.,  11.]]]])
Before flattening shape:  torch.Size([2, 1, 3, 2])
After flattening:  tensor([[  0.,   1.,   2.,   3.,   4.,   5.],
        [  6.,   7.,   8.,   9.,  10.,  11.]])
After flattening shape:  torch.Size([2, 6])


### Barebones PyTorch: Two-Layer Network

Here we define a function `two_layer_fc` which performs the forward pass of a two-layer fully-connected ReLU network on a batch of image data. After defining the forward pass we check that it doesn't crash and that it produces outputs of the right shape by running zeros through the network.

You don't have to write any code here, but it's important that you read and understand the implementation.

In [5]:
import torch.nn.functional as F  # useful stateless functions

def two_layer_fc(x, params):
    """
    A fully-connected neural networks; the architecture is:
    NN is fully connected -> ReLU -> fully connected layer.
    Note that this function only defines the forward pass; 
    PyTorch will take care of the backward pass for us.
    
    The input to the network will be a minibatch of data, of shape
    (N, d1, ..., dM) where d1 * ... * dM = D. The hidden layer will have H units,
    and the output layer will produce scores for C classes.
    
    Inputs:
    - x: A PyTorch Tensor of shape (N, d1, ..., dM) giving a minibatch of
      input data.
    - params: A list [w1, w2] of PyTorch Tensors giving weights for the network;
      w1 has shape (D, H) and w2 has shape (H, C).
    
    Returns:
    - scores: A PyTorch Tensor of shape (N, C) giving classification scores for
      the input data x.
    """
    # first we flatten the image
    x = flatten(x)  # shape: [batch_size, C x H x W]
    
    w1, w2 = params
    
    # Forward pass: compute predicted y using operations on Tensors. Since w1 and
    # w2 have requires_grad=True, operations involving these Tensors will cause
    # PyTorch to build a computational graph, allowing automatic computation of
    # gradients. Since we are no longer implementing the backward pass by hand we
    # don't need to keep references to intermediate values.
    # you can also use `.clamp(min=0)`, equivalent to F.relu()
    x = F.relu(x.mm(w1))
    x = x.mm(w2)
    return x
    

def two_layer_fc_test():
    hidden_layer_size = 42
    x = torch.zeros((64, 50), dtype=dtype)  # minibatch size 64, feature dimension 50
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    scores = two_layer_fc(x, [w1, w2])
    print(scores.size())  # you should see [64, 10]

two_layer_fc_test()

torch.Size([64, 10])


### Barebones PyTorch: Three-Layer ConvNet

Here you will complete the implementation of the function `three_layer_convnet`, which will perform the forward pass of a three-layer convolutional network. Like above, we can immediately test our implementation by passing zeros through the network. The network should have the following architecture:

1. A convolutional layer (with bias) with `channel_1` filters, each with shape `KW1 x KH1`, and zero-padding of two
2. ReLU nonlinearity
3. A convolutional layer (with bias) with `channel_2` filters, each with shape `KW2 x KH2`, and zero-padding of one
4. ReLU nonlinearity
5. Fully-connected layer with bias, producing scores for C classes.

**HINT**: For convolutions: http://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d; pay attention to the shapes of convolutional filters!

In [6]:
def three_layer_convnet(x, params):
    """
    Performs the forward pass of a three-layer convolutional network with the
    architecture defined above.

    Inputs:
    - x: A PyTorch Tensor of shape (N, 3, H, W) giving a minibatch of images
    - params: A list of PyTorch Tensors giving the weights and biases for the
      network; should contain the following:
      - conv_w1: PyTorch Tensor of shape (channel_1, 3, KH1, KW1) giving weights
        for the first convolutional layer
      - conv_b1: PyTorch Tensor of shape (channel_1,) giving biases for the first
        convolutional layer
      - conv_w2: PyTorch Tensor of shape (channel_2, channel_1, KH2, KW2) giving
        weights for the second convolutional layer
      - conv_b2: PyTorch Tensor of shape (channel_2,) giving biases for the second
        convolutional layer
      - fc_w: PyTorch Tensor giving weights for the fully-connected layer. Can you
        figure out what the shape should be?
      - fc_b: PyTorch Tensor giving biases for the fully-connected layer. Can you
        figure out what the shape should be?
    
    Returns:
    - scores: PyTorch Tensor of shape (N, C) giving classification scores for x
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    ################################################################################
    # TODO: Implement the forward pass for the three-layer ConvNet.                #
    ################################################################################
    N = x.shape[0]
#     print('N = ', N)
    
    h1 = F.conv2d(x, conv_w1, conv_b1, padding=2)
    f1 = F.relu(h1)
    h2 = F.conv2d(f1, conv_w2, conv_b2, padding=1)
    f2 = F.relu(h2)
#     print('f2.shape = ', f2.shape, f2.view(N, -1).shape)
#     f3 = F.linear(f2.view(N, -1), fc_w, fc_b)
    f3 = f2.view(N, -1).mm(fc_w.t()) + fc_b
    
    scores = f3
       
    
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################
    return scores

After defining the forward pass of the ConvNet above, run the following cell to test your implementation.

When you run this function, scores should have shape (64, 10).

In [7]:
def three_layer_convnet_test():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]

    conv_w1 = torch.zeros((6, 3, 5, 5), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b1 = torch.zeros((6,))  # out_channel
    conv_w2 = torch.zeros((9, 6, 3, 3), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b2 = torch.zeros((9,))  # out_channel

    # you must calculate the shape of the tensor after two conv layers, before the fully-connected layer
    fc_w = torch.zeros((10, 9 * 32 * 32))
    fc_b = torch.zeros(10)

    scores = three_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b])
    print(scores.size())  # you should see [64, 10]
#     print(scores)
three_layer_convnet_test()

torch.Size([64, 10])


### Barebones PyTorch: Initialization
Let's write a couple utility methods to initialize the weight matrices for our models.

- `random_weight(shape)` initializes a weight tensor with the Kaiming normalization method.
- `zero_weight(shape)` initializes a weight tensor with all zeros. Useful for instantiating bias parameters.

The `random_weight` function uses the Kaiming normal initialization method, described in:

He et al, *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [8]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator. 
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU. 
# Otherwise it should be `torch.FloatTensor`
print(random_weight((3, 5)))
print(zero_weight((2,)))


tensor([[ 0.6638, -0.3383,  0.6441,  0.5219,  1.0060],
        [-0.0102,  0.0093,  0.5817, -0.9320, -0.1924],
        [-0.5790, -0.5447, -0.3030,  0.4313,  1.2056]])
tensor([ 0.,  0.])


### Barebones PyTorch: Check Accuracy
When training the model we will use the following function to check the accuracy of our model on the training or validation sets.

When checking accuracy we don't need to compute any gradients; as a result we don't need PyTorch to build a computational graph for us when we compute scores. To prevent a graph from being built we scope our computation under a `torch.no_grad()` context manager.

In [9]:
def check_accuracy_part2(loader, model_fn, params):
    """
    Check the accuracy of a classification model.
    
    Inputs:
    - loader: A DataLoader for the data split we want to check
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model
    
    Returns: Nothing, but prints the accuracy of the model
    """
    split = 'val' if loader.dataset.train else 'test'
    print('Checking accuracy on the %s set' % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.int64)
            scores = model_fn(x, params)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

### BareBones PyTorch: Training Loop
We can now set up a basic training loop to train our network. We will train the model using stochastic gradient descent without momentum. We will use `torch.functional.cross_entropy` to compute the loss; you can [read about it here](http://pytorch.org/docs/stable/nn.html#cross-entropy).

The training loop takes as input the neural network function, a list of initialized parameters (`[w1, w2]` in our example), and learning rate.

In [18]:
def train_part2(model_fn, params, learning_rate):
    """
    Train a model on CIFAR-10.
    
    Inputs:
    - model_fn: A Python function that performs the forward pass of the model.
      It should have the signature scores = model_fn(x, params) where x is a
      PyTorch Tensor of image data, params is a list of PyTorch Tensors giving
      model weights, and scores is a PyTorch Tensor of shape (N, C) giving
      scores for the elements in x.
    - params: List of PyTorch Tensors giving weights for the model
    - learning_rate: Python scalar giving the learning rate to use for SGD
    
    Returns: Nothing
    """
    for t, (x, y) in enumerate(loader_train):
        # Move the data to the proper device (GPU or CPU)
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # Forward pass: compute scores and loss
        scores = model_fn(x, params)
        print('scores = ', scores[0:5])
        print('y = ', y[0:5])
        loss = F.cross_entropy(scores, y)

        # Backward pass: PyTorch figures out which Tensors in the computational
        # graph has requires_grad=True and uses backpropagation to compute the
        # gradient of the loss with respect to these Tensors, and stores the
        # gradients in the .grad attribute of each Tensor.
        loss.backward()

        # Update parameters. We don't want to backpropagate through the
        # parameter updates, so we scope the updates under a torch.no_grad()
        # context manager to prevent a computational graph from being built.
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad

                # Manually zero the gradients after running the backward pass
                w.grad.zero_()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part2(loader_val, model_fn, params)
            print()

### BareBones PyTorch: Train a Two-Layer Network
Now we are ready to run the training loop. We need to explicitly allocate tensors for the fully connected weights, `w1` and `w2`. 

Each minibatch of CIFAR has 64 examples, so the tensor shape is `[64, 3, 32, 32]`. 

After flattening, `x` shape should be `[64, 3 * 32 * 32]`. This will be the size of the first dimension of `w1`. 
The second dimension of `w1` is the hidden layer size, which will also be the first dimension of `w2`. 

Finally, the output of the network is a 10-dimensional vector that represents the probability distribution over 10 classes. 

You don't need to tune any hyperparameters but you should see accuracies above 40% after training for one epoch.

In [11]:
hidden_layer_size = 4000
learning_rate = 1e-2

w1 = random_weight((3 * 32 * 32, hidden_layer_size))
w2 = random_weight((hidden_layer_size, 10))

train_part2(two_layer_fc, [w1, w2], learning_rate)

Iteration 0, loss = 3.8256
Checking accuracy on the val set
Got 126 / 1000 correct (12.60%)

Iteration 100, loss = 2.6318
Checking accuracy on the val set
Got 316 / 1000 correct (31.60%)

Iteration 200, loss = 1.9601
Checking accuracy on the val set
Got 355 / 1000 correct (35.50%)

Iteration 300, loss = 1.6091
Checking accuracy on the val set
Got 407 / 1000 correct (40.70%)

Iteration 400, loss = 1.6400
Checking accuracy on the val set
Got 407 / 1000 correct (40.70%)

Iteration 500, loss = 1.4945
Checking accuracy on the val set
Got 375 / 1000 correct (37.50%)

Iteration 600, loss = 1.6579
Checking accuracy on the val set
Got 425 / 1000 correct (42.50%)

Iteration 700, loss = 1.4376
Checking accuracy on the val set
Got 437 / 1000 correct (43.70%)



### BareBones PyTorch: Training a ConvNet

In the below you should use the functions defined above to train a three-layer convolutional network on CIFAR. The network should have the following architecture:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

You should initialize your weight matrices using the `random_weight` function defined above, and you should initialize your bias vectors using the `zero_weight` function above.

You don't need to tune any hyperparameters, but if everything works correctly you should achieve an accuracy above 42% after one epoch.

In [ ]:
learning_rate = 3e-3


# channel_1 = 32
# channel_2 = 16

channel_1 = 64
channel_2 = 32


conv_w1 = None
conv_b1 = None
conv_w2 = None
conv_b2 = None
fc_w = None
fc_b = None

################################################################################
# TODO: Initialize the parameters of a three-layer ConvNet.                    #
################################################################################

conv_w1 = random_weight((channel_1, 3, 5, 5))
conv_b1 = zero_weight((channel_1))

conv_w2 = random_weight((channel_2, channel_1, 3, 3))
conv_b2 = zero_weight((channel_2))

fc_w = random_weight((10, 32768))
fc_b = zero_weight((10))

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
train_part2(three_layer_convnet, params, learning_rate)

scores =  tensor([[ -98.4058,  -94.9369,  -50.9909,   75.5732, -102.0989,  -87.2860,
         -126.8824,   34.8531,   28.5550,   32.0790],
        [ -64.3196,  -75.6984,  -58.0056,  -64.0795,   -0.6000,   33.8041,
          -98.8529,  -23.1254,   66.1724,   91.7237],
        [ -60.1062,   -6.6062,  -22.8888,   -0.8940,   23.7145,  -18.8316,
         -105.4350,  -44.4281,   39.4973,   49.5351],
        [ -67.8866,  -21.4708,  -16.2878,   54.8781,  -39.6928,  -74.1574,
          -29.8957,   17.3433,  -10.7631,   51.9379],
        [ -79.9790,  -27.7298,  -44.7889,   -7.5417,  -39.6825,  -20.7898,
          -60.6228,  -16.5255,    8.3067,  -39.2525]])
y =  tensor([ 1,  9,  6,  6,  4])
Iteration 0, loss = 122.3860
Checking accuracy on the val set
Got 118 / 1000 correct (11.80%)

scores =  tensor([[  77.2345,   28.3769,  -35.7725,   61.0529, -194.4277, -178.9704,
          -96.7084, -120.7314,   22.0829,  -78.1551],
        [ -80.9749,   32.1972, -111.1881,   17.3174, -168.1488, -111.4778,
 

scores =  tensor([[-1.1489,  0.5139,  0.0703, -0.1613, -0.1688, -1.0389,  0.8764,
         -0.6847,  0.1747,  0.7932],
        [ 0.0042, -0.0987, -0.8308, -0.1001,  0.1790, -1.0255, -0.2131,
          0.6299, -0.3283,  0.4804],
        [ 0.6542, -0.0269, -0.5793, -0.7226,  1.0606,  0.0355,  1.2990,
          1.1554,  1.1730, -0.4549],
        [ 0.7248,  0.1043, -0.1531, -1.9825, -0.3175, -0.5579, -1.4919,
         -0.3090, -0.1220,  0.4056],
        [-0.7867, -1.3733,  0.8848,  0.7736, -0.6683,  0.4176,  0.1720,
         -0.5048, -1.4061, -0.1784]])
y =  tensor([ 6,  4,  5,  8,  8])
scores =  tensor([[-0.8446,  0.7136,  0.5997, -1.2175, -1.8036, -0.7154,  0.1431,
          1.2900, -0.2709,  2.0631],
        [-0.8394,  0.9257, -0.1991,  0.8287,  1.2383, -0.2916,  0.4099,
         -0.1026, -0.0507, -0.1692],
        [ 0.3242,  0.7038,  1.0214, -1.0754,  0.7684, -0.2363,  0.8201,
         -0.8153,  0.0276,  1.1839],
        [-0.7593, -0.3798, -1.0419,  0.4884, -1.5991,  0.3731, -1.1432,
 

scores =  tensor([[ 0.3860,  0.1420, -0.3988, -0.8756, -1.1802,  0.1585, -0.3445,
         -0.3839,  0.9386,  0.2291],
        [-0.2796, -0.2685,  1.1420,  0.7508,  0.3964, -0.2370,  0.2736,
         -0.0314,  0.0685, -0.0048],
        [-0.8303, -0.0500, -1.1198,  0.6793,  0.2468,  0.0225, -0.1077,
         -0.2759, -0.5109,  0.0252],
        [ 0.0105, -0.3359, -0.0074,  0.3899,  0.3048, -0.5244,  0.0562,
          0.7144,  0.1560, -0.1970],
        [-0.3978,  0.6556, -0.5190, -0.3048, -0.4803,  0.1388,  0.5420,
         -0.4775,  0.1218, -1.4922]])
y =  tensor([ 3,  6,  7,  2,  8])
scores =  tensor([[-0.6153,  0.9570,  0.1974, -0.7970, -0.9110, -0.3585, -0.0390,
          0.2308,  0.3416, -0.2356],
        [-0.3133,  0.5220, -0.1268,  0.3069,  0.7763, -0.3349, -0.4447,
          0.3719, -0.1432,  0.0414],
        [ 0.2822, -0.1304,  0.0262, -1.7793, -0.2425,  0.4606, -0.5803,
          0.7834, -0.4081,  2.4876],
        [-0.9340, -0.6564, -0.0721,  0.2303,  0.2638,  0.1535,  0.3931,
 

scores =  tensor([[-0.5877,  0.2294, -1.3998,  0.7679, -0.4823, -0.4003,  0.5954,
         -0.7642, -1.0942,  0.5014],
        [ 0.2288,  0.2258, -0.8601, -0.6174, -0.2346, -0.3589,  0.4338,
         -0.7329,  0.9641,  0.0694],
        [-0.5555, -0.2638, -0.5942, -0.1100, -1.0508,  0.1885,  0.0780,
         -0.1771,  0.4239,  0.3364],
        [-0.4536, -0.0874,  0.1290,  0.1073, -0.1003, -1.0559,  0.3663,
          0.3947,  0.4099,  0.1230],
        [ 0.1564,  1.2049, -0.7805, -0.5737,  1.8057, -0.4776, -1.6847,
         -1.7085,  0.0277,  0.3100]])
y =  tensor([ 6,  8,  0,  7,  1])
scores =  tensor([[ 0.0257, -0.3500, -0.0125, -0.2111, -0.2010, -0.2746, -0.1653,
         -0.1194, -0.0572,  0.2862],
        [ 0.6160,  0.1165, -0.0098, -0.0060, -0.3084,  0.4403,  0.6839,
         -0.6003,  0.2664, -0.2405],
        [ 0.0191, -0.0651, -0.1670, -0.2818, -0.0930,  0.0448,  0.0129,
          0.2684, -0.0142, -0.0804],
        [-0.1642,  0.5004, -0.2920,  0.0579, -0.2420,  0.4601,  0.5519,
 

scores =  tensor([[-0.2111, -0.0829, -0.0624, -0.2363, -0.0636,  0.0286, -0.1519,
         -0.0995,  0.0475,  0.0852],
        [-0.2106, -0.2483, -0.0581,  0.0308, -0.3123, -0.1993,  0.3437,
         -0.1961,  0.0672,  0.0137],
        [ 0.6567,  0.4603, -0.3551, -0.7144, -1.2276,  0.4146,  0.2583,
         -0.7127,  0.8258, -0.4114],
        [ 0.1142,  0.4706,  0.2450, -0.1557, -0.4111, -0.0063,  1.0446,
         -0.1942, -0.6393, -0.1848],
        [ 0.2077,  0.0690, -0.0833, -0.0206,  0.0215, -0.0648, -0.0135,
         -0.0981,  0.0905, -0.0944]])
y =  tensor([ 2,  1,  8,  3,  0])
scores =  tensor([[ 0.0357,  0.5254, -0.4710,  0.2161, -0.3824,  0.2877, -0.1226,
          0.3627,  0.2377,  0.3816],
        [ 0.1319,  0.6236, -1.0436, -0.2013,  1.4142, -0.5975, -1.0288,
          2.1443,  1.0533,  1.2116],
        [-0.0110,  0.9266, -0.8048, -0.4961, -0.7956,  0.0512,  0.6685,
         -0.3544,  1.1192,  0.7814],
        [-0.0250,  0.0718,  0.0489,  0.0265,  0.0935, -0.1966,  0.3312,
 

...

scores =  tensor([[ 0.3546, -0.0691, -0.8566,  0.6592, -0.2651, -0.1629, -0.5923,
          0.4963, -0.5320,  0.1075],
        [ 0.9615, -0.3694, -0.5535, -0.2227, -0.5516, -0.4942, -0.1508,
         -0.9582,  0.6997, -0.1828],
        [ 0.0702, -0.6325,  0.0645,  0.3739,  0.1983,  0.0944, -0.7626,
          0.2353,  0.1710, -0.6049],
        [-0.1102, -0.0029,  0.2186,  0.3292,  0.1974, -0.2383, -0.1257,
         -0.0528, -0.0308, -0.0352],
        [ 0.5432,  0.0215, -0.9570,  0.1011, -1.3997,  0.2490, -0.2288,
         -0.2112,  0.4911,  0.8591]])
y =  tensor([ 4,  9,  3,  1,  9])
scores =  tensor([[-0.0896, -0.0522, -0.0035,  0.1028,  0.1183,  0.1019, -0.0483,
         -0.0670,  0.0162,  0.0449],
        [-0.0134, -0.0040,  0.0094,  0.0125,  0.0147,  0.0035, -0.0015,
          0.0073, -0.0168, -0.0118],
        [ 0.4398, -0.3794, -0.4660, -0.0098, -0.4216, -0.1870, -0.0387,
         -0.3659,  0.3223,  0.1120],
        [ 0.8708, -0.2675, -1.3317, -1.1268, -1.2285,  0.4002, -0.2519,
 

scores =  tensor([[-0.0244,  0.0321,  0.0102, -0.0677,  0.0130,  0.0541,  0.0603,
          0.0170, -0.0194, -0.0081],
        [ 0.0975, -0.0202, -0.2687, -0.2873, -0.5562,  0.1456,  0.0680,
          0.1955,  0.2629,  0.6228],
        [ 0.3857, -0.4132, -0.1819, -0.2055, -0.5276,  0.0062, -0.2890,
         -0.1090,  0.6151,  0.8517],
        [-0.0707, -0.1629, -0.2284,  0.1381, -0.0358, -0.0752, -0.0580,
         -0.1563,  0.1064,  0.1830],
        [-0.1541,  0.2720,  0.0835, -0.2516, -0.0676,  0.1180,  0.0807,
          0.2051,  0.5103,  0.2969]])
y =  tensor([ 2,  8,  9,  5,  8])
scores =  tensor([[ 1.1707,  0.0064, -1.4737, -1.9361, -1.2307, -0.4413, -0.2614,
         -0.2597,  2.0714,  0.8528],
        [ 0.1301, -0.1346,  0.4085,  0.1280, -0.1226,  0.0535,  0.2494,
         -0.0636, -0.1193,  0.1140],
        [-0.0030,  0.0845, -0.1098,  0.0563,  0.0296, -0.2344, -0.0778,
         -0.0425, -0.2021, -0.0591],
        [-0.0088, -0.3850, -0.1878, -0.4771, -0.4887,  0.1386,  0.0232,
 

scores =  tensor([[-0.0216, -0.0070, -0.0089,  0.0373,  0.0106,  0.0059, -0.0068,
         -0.0042, -0.0271, -0.0266],
        [ 0.3666, -0.0787, -0.6915,  0.1903, -0.9533,  0.4218, -0.3629,
         -0.4383,  0.3051, -0.1832],
        [ 0.1023,  0.1057, -0.2101, -0.3235, -0.0699, -0.0804,  0.0830,
          0.1208,  0.1971,  0.0313],
        [-0.0153,  0.0005,  0.0099,  0.0047,  0.0150,  0.0093,  0.0062,
          0.0085, -0.0198, -0.0115],
        [ 1.1354,  0.9971, -1.9048, -1.6710, -0.4214, -0.5677,  0.4058,
         -0.6525,  2.2875,  1.2236]])
y =  tensor([ 0,  8,  1,  4,  9])
scores =  tensor([[-0.0075,  0.1863, -0.1602,  0.0165, -0.0569, -0.2724,  0.0449,
         -0.0273, -0.3532,  0.0157],
        [ 0.2513,  0.3997,  0.3270,  0.4507, -0.2430,  0.0321, -0.4875,
          0.4503, -0.0084,  0.2588],
        [ 1.0235,  0.2776, -0.0684, -1.0491, -0.6623, -0.2687, -1.0011,
         -0.2293,  0.5759,  0.7361],
        [-0.0562,  0.0159,  0.0096,  0.0543,  0.0384,  0.0050, -0.0356,
 

scores =  tensor([[ 0.2246, -0.1207,  0.2308,  0.1074, -0.1084,  0.2231,  1.0217,
         -0.2977, -0.3687, -0.1015],
        [-0.0460, -0.0177,  0.0448,  0.0553,  0.0039, -0.0058,  0.0417,
          0.0213, -0.0254, -0.0657],
        [ 0.4214,  0.2678,  0.2306, -0.1298,  0.0975,  0.0022, -0.2191,
         -0.1057,  0.3848,  0.3081],
        [ 0.0399,  0.7807, -0.2465, -0.1226,  0.1791,  0.1242,  0.2668,
          0.0125,  0.0390,  0.6923],
        [-0.0013, -0.1810, -0.1356, -0.2117, -0.7121, -0.3466,  0.0374,
         -0.0601,  0.0508,  0.2811]])
y =  tensor([ 6,  6,  9,  2,  1])
scores =  tensor([[-0.7892,  0.1691, -0.0403, -0.1209, -0.0708,  0.1936,  0.2201,
         -0.0929,  0.0371, -0.2172],
        [ 0.3196,  0.0792,  0.1197, -0.5038, -0.2311, -0.2017,  0.4280,
         -0.0912,  0.3243,  0.5545],
        [-0.0311, -0.1422, -0.0221, -0.6004, -0.3034,  0.4061, -0.1623,
          0.4731,  0.0036, -0.0229],
        [-0.2610, -0.3788, -0.6429,  0.7609, -0.7650, -0.2085,  0.1938,
 

scores =  tensor([[-0.1042, -0.0887,  0.0762,  0.1411, -0.0252,  0.2399,  0.1527,
          0.3034, -0.0891,  0.1273],
        [-0.0525,  0.0407, -0.1393, -0.1393,  0.0887,  0.1952,  0.1470,
          0.0746,  0.2413, -0.1520],
        [ 1.9751, -0.3222, -0.3284, -0.2465, -0.1828, -0.0894, -0.2354,
         -0.4831,  1.1956, -0.2133],
        [-0.0998,  0.0753,  0.0569,  0.2388, -0.1036, -0.1063,  0.3064,
          0.1661, -0.2792, -0.2173],
        [ 0.0113, -0.0230,  0.0372, -0.0609, -0.0203, -0.0312,  0.0706,
          0.0970, -0.0871,  0.1190]])
y =  tensor([ 5,  6,  0,  1,  4])
scores =  tensor([[-0.1080, -0.2063,  0.1063, -0.2198, -0.1828, -0.0571, -0.0284,
          0.2239, -0.0592,  0.1678],
        [-0.3709,  0.1743, -0.4732, -0.1266, -0.4248,  0.2899,  0.3877,
         -0.1544, -0.1814,  0.0224],
        [-0.0393,  0.0428,  0.1334, -0.0018,  0.0419,  0.0585,  0.0229,
          0.0767,  0.0587,  0.1617],
        [ 0.1203, -0.0728,  0.0253, -0.0354,  0.0545,  0.0761, -0.0620,
 

# Part III. PyTorch Module API

Barebone PyTorch requires that we track all the parameter tensors by hand. This is fine for small networks with a few tensors, but it would be extremely inconvenient and error-prone to track tens or hundreds of tensors in larger networks.

PyTorch provides the `nn.Module` API for you to define arbitrary network architectures, while tracking every learnable parameters for you. In Part II, we implemented SGD ourselves. PyTorch also provides the `torch.optim` package that implements all the common optimizers, such as RMSProp, Adagrad, and Adam. It even supports approximate second-order methods like L-BFGS! You can refer to the [doc](http://pytorch.org/docs/master/optim.html) for the exact specifications of each optimizer.

To use the Module API, follow the steps below:

1. Subclass `nn.Module`. Give your network class an intuitive name like `TwoLayerFC`. 

2. In the constructor `__init__()`, define all the layers you need as class attributes. Layer objects like `nn.Linear` and `nn.Conv2d` are themselves `nn.Module` subclasses and contain learnable parameters, so that you don't have to instantiate the raw tensors yourself. `nn.Module` will track these internal parameters for you. Refer to the [doc](http://pytorch.org/docs/master/nn.html) to learn more about the dozens of builtin layers. **Warning**: don't forget to call the `super().__init__()` first!

3. In the `forward()` method, define the *connectivity* of your network. You should use the attributes defined in `__init__` as function calls that take tensor as input and output the "transformed" tensor. Do *not* create any new layers with learnable parameters in `forward()`! All of them must be declared upfront in `__init__`. 

After you define your Module subclass, you can instantiate it as an object and call it just like the NN forward function in part II.

### Module API: Two-Layer Network
Here is a concrete example of a 2-layer fully connected network:

In [ ]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        # assign layer objects to class attributes
        self.fc1 = nn.Linear(input_size, hidden_size)
        # nn.init package contains convenient initialization methods
        # http://pytorch.org/docs/master/nn.html#torch-nn-init 
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)
    
    def forward(self, x):
        # forward always defines connectivity
        x = flatten(x)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

def test_TwoLayerFC():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # minibatch size 64, feature dimension 50
    model = TwoLayerFC(input_size, 42, 10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_TwoLayerFC()

### Module API: Three-Layer ConvNet
It's your turn to implement a 3-layer ConvNet followed by a fully connected layer. The network architecture should be the same as in Part II:

1. Convolutional layer with `channel_1` 5x5 filters with zero-padding of 2
2. ReLU
3. Convolutional layer with `channel_2` 3x3 filters with zero-padding of 1
4. ReLU
5. Fully-connected layer to `num_classes` classes

You should initialize the weight matrices of the model using the Kaiming normal initialization method.

**HINT**: http://pytorch.org/docs/stable/nn.html#conv2d

After you implement the three-layer ConvNet, the `test_ThreeLayerConvNet` function will run your implementation; it should print `(64, 10)` for the shape of the output scores.

In [ ]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        pass
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        pass
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


def test_ThreeLayerConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_ThreeLayerConvNet()

### Module API: Check Accuracy
Given the validation or test set, we can check the classification accuracy of a neural network. 

This version is slightly different from the one in part II. You don't manually pass in the parameters anymore.

In [ ]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

### Module API: Training Loop
We also use a slightly different training loop. Rather than updating the values of the weights ourselves, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [ ]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

### Module API: Train a Two-Layer Network
Now we are ready to run the training loop. In contrast to part II, we don't explicitly allocate parameter tensors anymore.

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `TwoLayerFC`. 

You also need to define an optimizer that tracks all the learnable parameters inside `TwoLayerFC`.

You don't need to tune any hyperparameters, but you should see model accuracies above 40% after training for one epoch.

In [ ]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

### Module API: Train a Three-Layer ConvNet
You should now use the Module API to train a three-layer ConvNet on CIFAR. This should look very similar to training the two-layer network! You don't need to tune any hyperparameters, but you should achieve above above 45% after training for one epoch.

You should train the model using stochastic gradient descent without momentum.

In [ ]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = None
optimizer = None
################################################################################
# TODO: Instantiate your ThreeLayerConvNet model and a corresponding optimizer #
################################################################################
pass
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer)

# Part IV. PyTorch Sequential API

Part III introduced the PyTorch Module API, which allows you to define arbitrary learnable layers and their connectivity. 

For simple models like a stack of feed forward layers, you still need to go through 3 steps: subclass `nn.Module`, assign layers to class attributes in `__init__`, and call each layer one by one in `forward()`. Is there a more convenient way? 

Fortunately, PyTorch provides a container Module called `nn.Sequential`, which merges the above steps into one. It is not as flexible as `nn.Module`, because you cannot specify more complex topology than a feed-forward stack, but it's good enough for many use cases.

### Sequential API: Two-Layer Network
Let's see how to rewrite our two-layer fully connected network example with `nn.Sequential`, and train it using the training loop defined above.

Again, you don't need to tune any hyperparameters here, but you shoud achieve above 40% accuracy after one epoch of training.

In [ ]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

hidden_layer_size = 4000
learning_rate = 1e-2

model = nn.Sequential(
    Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# you can use Nesterov momentum in optim.SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model, optimizer)

### Sequential API: Three-Layer ConvNet
Here you should use `nn.Sequential` to define and train a three-layer ConvNet with the same architecture we used in Part III:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

You should initialize your weight matrices using the `random_weight` function defined above, and you should initialize your bias vectors using the `zero_weight` function above.

You should optimize your model using stochastic gradient descent with Nesterov momentum 0.9.

Again, you don't need to tune any hyperparameters but you should see accuracy above 55% after one epoch of training.

In [ ]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

################################################################################
# TODO: Rewrite the 2-layer ConvNet with bias from Part III with the           #
# Sequential API.                                                              #
################################################################################
pass
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer)

# Part V. CIFAR-10 open-ended challenge

In this section, you can experiment with whatever ConvNet architecture you'd like on CIFAR-10. 

Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves **at least 70%** accuracy on the CIFAR-10 **validation** set within 10 epochs. You can use the check_accuracy and train functions from above. You can use either `nn.Module` or `nn.Sequential` API. 

Describe what you did at the end of this notebook.

Here are the official API documentation for each component. One note: what we call in the class "spatial batch norm" is called "BatchNorm2D" in PyTorch.

* Layers in torch.nn package: http://pytorch.org/docs/stable/nn.html
* Activations: http://pytorch.org/docs/stable/nn.html#non-linear-activations
* Loss functions: http://pytorch.org/docs/stable/nn.html#loss-functions
* Optimizers: http://pytorch.org/docs/stable/optim.html


### Things you might try:
- **Filter size**: Above we used 5x5; would smaller filters be more efficient?
- **Number of filters**: Above we used 32 filters. Do more or fewer do better?
- **Pooling vs Strided Convolution**: Do you use max pooling or just stride convolutions?
- **Batch normalization**: Try adding spatial batch normalization after convolution layers and vanilla batch normalization after affine layers. Do your networks train faster?
- **Network architecture**: The network above has two layers of trainable parameters. Can you do better with a deep network? Good architectures to try include:
    - [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]
- **Global Average Pooling**: Instead of flattening and then having multiple affine layers, perform convolutions until your image gets small (7x7 or so) and then perform an average pooling operation to get to a 1x1 image picture (1, 1 , Filter#), which is then reshaped into a (Filter#) vector. This is used in [Google's Inception Network](https://arxiv.org/abs/1512.00567) (See Table 1 for their architecture).
- **Regularization**: Add l2 weight regularization, or perhaps use Dropout.

### Tips for training
For each network architecture that you try, you should tune the learning rate and other hyperparameters. When doing this there are a couple important things to keep in mind:

- If the parameters are working well, you should see improvement within a few hundred iterations
- Remember the coarse-to-fine approach for hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all.
- Once you have found some sets of parameters that seem to work, search more finely around these parameters. You may need to train for more epochs.
- You should use the validation set for hyperparameter search, and save your test set for evaluating your architecture on the best parameters as selected by the validation set.

### Going above and beyond
If you are feeling adventurous there are many other features you can implement to try and improve your performance. You are **not required** to implement any of these, but don't miss the fun if you have time!

- Alternative optimizers: you can try Adam, Adagrad, RMSprop, etc.
- Alternative activation functions such as leaky ReLU, parametric ReLU, ELU, or MaxOut.
- Model ensembles
- Data augmentation
- New Architectures
  - [ResNets](https://arxiv.org/abs/1512.03385) where the input from the previous layer is added to the output.
  - [DenseNets](https://arxiv.org/abs/1608.06993) where inputs into previous layers are concatenated together.
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

### Have fun and happy training! 

In [ ]:
################################################################################
# TODO:                                                                        #         
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################
model = None
optimizer = None

pass
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

# You should get at least 70% accuracy
train_part34(model, optimizer, epochs=10)

## Describe what you did 

In the cell below you should write an explanation of what you did, any additional features that you implemented, and/or any graphs that you made in the process of training and evaluating your network.

TODO: Describe what you did

## Test set -- run this only once

Now that we've gotten a result we're happy with, we test our final model on the test set (which you should store in best_model). Think about how this compares to your validation set accuracy.

In [ ]:
best_model = model
check_accuracy_part34(loader_test, best_model)